In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
from datetime import datetime, timedelta
np.random.seed(1)

In [14]:
ks_data = pd.read_csv('ks_overall_dataset.csv')
ks_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200232 entries, 0 to 200231
Data columns (total 27 columns):
blurb                       200230 non-null object
category_name               200232 non-null object
category_specific           200232 non-null object
country_displayable_name    200232 non-null object
created_at                  200232 non-null int64
date_created                200232 non-null object
deadline                    200232 non-null int64
date_deadline               200232 non-null object
disable_communication       200232 non-null bool
id                          200232 non-null int64
is_starrable                200232 non-null bool
launched_at                 200232 non-null int64
date_launched               200232 non-null object
date_launched_year          200232 non-null int64
date_launched_month         200232 non-null int64
location_name               200025 non-null object
name                        200232 non-null object
photo_link                  2002

In [5]:
# 테스트 이미지 셋 크롤링
import time
from selenium import webdriver
import requests
img_path = './test_image'
for idx, link in enumerate(ks_data['photo_link'][:100]):
    try:
        img = requests.get(link).content
        with open(img_path+'/'+str(idx)+".png",'wb') as f:
            f.write(img)
    except:
        print(idx)
        print(link)
        pass

In [ ]:
# index 번호 추출 참고 코드
from glob import glob

image_list = glob('./image_2015_2019/*.png')
path = image_list[0]
print(path)
#./image_2015_2019/72671.png

def get_label_from_path(path):
    return int(path.split('/')[-1].split('.')[0])
#"./image_2015_2019/72671.png".split('/')[-1].split('.')[0]
#'72671'

path = image_list[9999]
path, get_label_from_path(path)
#('./image_2015_2019/45352.png', 45352)

In [6]:
# 디렉토리에서 이미지와 index번호 가져오기 시도
from PIL import Image
import glob
image_list = []
#path = './test_image/{}.png'.format()

for filename in glob.glob('./test_image/*.png'):
    index = filename.split('/')[-1].split('\\')[1].split('.')[0]
    im = Image.open(filename)
    image_list.append([index, im])
    #images.append(np.asarray(im))

In [9]:
test_df = pd.DataFrame(image_list, columns=['index','image'])
test_df

index                                              image
0      0  <PIL.PngImagePlugin.PngImageFile image mode=RG...
1      1  <PIL.JpegImagePlugin.JpegImageFile image mode=...
2     10  <PIL.JpegImagePlugin.JpegImageFile image mode=...
3     11  <PIL.JpegImagePlugin.JpegImageFile image mode=...
4     12  <PIL.JpegImagePlugin.JpegImageFile image mode=...
..   ...                                                ...
95    95  <PIL.JpegImagePlugin.JpegImageFile image mode=...
96    96  <PIL.JpegImagePlugin.JpegImageFile image mode=...
97    97  <PIL.JpegImagePlugin.JpegImageFile image mode=...
98    98  <PIL.JpegImagePlugin.JpegImageFile image mode=...
99    99  <PIL.PngImagePlugin.PngImageFile image mode=RG...

[100 rows x 2 columns]

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
index    100 non-null object
image    100 non-null object
dtypes: object(2)
memory usage: 1.7+ KB


In [11]:
test_df['index'] = test_df['index'].astype('int64') #index 컬럼 --> int로 타입 변경

In [12]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
index    100 non-null int64
image    100 non-null object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [13]:
test_df.sort_values(by='index', inplace=True) #index에 맞춰 정렬

In [14]:
test_df

index                                              image
0       0  <PIL.PngImagePlugin.PngImageFile image mode=RG...
1       1  <PIL.JpegImagePlugin.JpegImageFile image mode=...
12      2  <PIL.JpegImagePlugin.JpegImageFile image mode=...
23      3  <PIL.JpegImagePlugin.JpegImageFile image mode=...
34      4  <PIL.JpegImagePlugin.JpegImageFile image mode=...
..    ...                                                ...
95     95  <PIL.JpegImagePlugin.JpegImageFile image mode=...
96     96  <PIL.JpegImagePlugin.JpegImageFile image mode=...
97     97  <PIL.JpegImagePlugin.JpegImageFile image mode=...
98     98  <PIL.JpegImagePlugin.JpegImageFile image mode=...
99     99  <PIL.PngImagePlugin.PngImageFile image mode=RG...

[100 rows x 2 columns]

In [15]:
test_df.reset_index(drop=True, inplace=True)

In [16]:
test_df

index                                              image
0       0  <PIL.PngImagePlugin.PngImageFile image mode=RG...
1       1  <PIL.JpegImagePlugin.JpegImageFile image mode=...
2       2  <PIL.JpegImagePlugin.JpegImageFile image mode=...
3       3  <PIL.JpegImagePlugin.JpegImageFile image mode=...
4       4  <PIL.JpegImagePlugin.JpegImageFile image mode=...
..    ...                                                ...
95     95  <PIL.JpegImagePlugin.JpegImageFile image mode=...
96     96  <PIL.JpegImagePlugin.JpegImageFile image mode=...
97     97  <PIL.JpegImagePlugin.JpegImageFile image mode=...
98     98  <PIL.JpegImagePlugin.JpegImageFile image mode=...
99     99  <PIL.PngImagePlugin.PngImageFile image mode=RG...

[100 rows x 2 columns]

In [17]:
test_df['image'][0]

In [ ]:
# 관건은 VGG16모델의 CNN을 사용하기 위해 분류 class가 꼭 필요한건지? 내가 원하는건 분류모델을 돌리기 전의 Flatten된 상태인데, 꼭 1, 0 따로 폴더 만들어 저장하고 해야 하는건지?

In [7]:
aa = np.array(image_list[0][1])
aa.shape

(315, 560, 4)

In [19]:
df_image = test_df['image']

In [22]:
df_image = pd.DataFrame(df_image)

In [24]:
df_image

image
0   <PIL.PngImagePlugin.PngImageFile image mode=RG...
1   <PIL.JpegImagePlugin.JpegImageFile image mode=...
2   <PIL.JpegImagePlugin.JpegImageFile image mode=...
3   <PIL.JpegImagePlugin.JpegImageFile image mode=...
4   <PIL.JpegImagePlugin.JpegImageFile image mode=...
..                                                ...
95  <PIL.JpegImagePlugin.JpegImageFile image mode=...
96  <PIL.JpegImagePlugin.JpegImageFile image mode=...
97  <PIL.JpegImagePlugin.JpegImageFile image mode=...
98  <PIL.JpegImagePlugin.JpegImageFile image mode=...
99  <PIL.PngImagePlugin.PngImageFile image mode=RG...

[100 rows x 1 columns]

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

In [4]:
conv_base.layers

In [5]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [6]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
# dataframe에서 이미지 로드 및 generator 생성
#datagen = ImageDataGenerator(rescale=1./255)
#generator = datagen.flow_from_dataframe(dataframe=df_image, target_size=(150,150), batch_size=32)

In [59]:
# 디렉토리에서 이미지 로드 및 generator 생성

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    """
    디렉토리로 부터 Image들을 읽어 들여 VGG16의 특성추출 레이어를 이용해 이미지 특성 추출
    directory: 이미지를 가져올 디렉토리
    sample_count: 학습시킬 이미지 개수 (이걸로 총 몇개의 이미지를 만들지)
    """
#   추출된 이미지 특성들을 저장할 ndarray변수.  array 크기: (개수, conv_base의 출력 shape)
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    filenames = []
    
    # ImageDataGenerator를 이용해 이미지 가져오기.
    generator = datagen.flow_from_directory(directory, #디렉토리로부터 이미지를 읽어와서
                                        target_size=(150, 150), #모든 이미지들 사이즈를 150, 150으로 맞출거다
                                        batch_size=batch_size, #한번에 몇장 만들지
                                        class_mode=None)
    
    i = 0 #predict한 이미지 수를 저장해 sample_count보다 커지면 break 한다.
    for inputs_batch in generator: #input data가 나온다
        #VGG16을 통해 나오는 특성맵을 저장
        feature_map_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = feature_map_batch #index
        
        #idx = (generator.batch_index - 1) * generator.batch_size
        #filenames = generator.filenames[idx : idx + generator.batch_size]
        #print(type(generator.filenames), len(generator.filenames), generator.filenames)
        i += 1
        
        if i * batch_size >= sample_count:
            break
    return features, generator.filenames #우리가 가져온 이미지들이 VGG16 layer를 통과하고 거쳐 나온 feature map들이 들어있다

In [84]:
directory = './test_image'
feature, filenames = extract_features(directory, 100)

Found 100 images belonging to 1 classes.


In [54]:
feature.shape

(100, 4, 4, 512)

In [55]:
train_data = feature.reshape(100, 4*4*512)

In [56]:
train_data.shape

(100, 8192)

In [85]:
df_image = pd.DataFrame(train_data)
#df_image['index'] = filenames
#df_image['index'] = df_image['index'].apply(lambda x: x.split('\\')[1].split('.')[0])
df_image['index'] = [filename.split('\\')[1].split('.')[0] for filename in filenames]
df_image['index'] = df_image['index'].astype('int64')
df_image = df_image.set_index('index').sort_index().reset_index()
df_image

index         0    1         2         3         4    5         6  \
0       0  1.219996  0.0  0.000000  0.000000  0.941779  0.0  0.000000   
1       1  0.172252  0.0  0.092992  0.000000  0.000000  0.0  0.368501   
2       2  0.253287  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3       3  0.290751  0.0  0.004097  0.000000  0.000000  0.0  0.000000   
4       4  0.087229  0.0  0.000000  0.000000  0.000895  0.0  0.000000   
..    ...       ...  ...       ...       ...       ...  ...       ...   
95     95  0.000000  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
96     96  0.704281  0.0  0.738247  0.000000  0.000000  0.0  0.000000   
97     97  0.133700  0.0  0.000000  0.136645  0.110879  0.0  0.000000   
98     98  0.797499  0.0  0.000000  0.000000  0.092572  0.0  0.000000   
99     99  0.601188  0.0  0.000000  0.000000  0.163435  0.0  0.000000   

           7    8  ...      8182      8183      8184      8185      8186  \
0   0.000000  0.0  ...  0.815925  0.071068  0.303899  0.000000  0.000000   
1   0.000000  0.0  ...  1.046614  0.296470  0.347764  0.000000  0.000000   
2   0.000000  0.0  ...  0.601681  0.227353  0.000000  0.000000  0.462870   
3   0.000000  0.0  ...  0.646326  1.059219  0.646514  1.171254  0.000000   
4   0.476952  0.0  ...  1.304575  1.134288  0.000000  0.000000  0.000000   
..       ...  ...  ...       ...       ...       ...       ...       ...   
95  0.000000  0.0  ...  0.731684  0.000000  0.000000  0.000000  0.000000   
96  0.055655  0.0  ...  0.729270  1.056348  0.000000  0.000000  0.000235   
97  0.000000  0.0  ...  1.228313  0.172605  0.307595  0.000000  0.000000   
98  0.287322  0.0  ...  1.818796  0.022135  0.000000  0.000000  0.000000   
99  0.000000  0.0  ...  0.974361  0.000000  0.336879  0.000000  0.000000   

        8187      8188      8189      8190      8191  
0   0.588848  0.000000  0.000000  0.494478  0.000000  
1   0.000000  0.000000  0.000000  0.275173  0.000000  
2   0.168930  0.297659  0.280102  0.092366  0.035608  
3   0.216076  0.000000  0.000000  0.000000  0.000000  
4   0.419175  0.000000  0.000000  0.741279  0.000000  
..       ...       ...       ...       ...       ...  
95  0.000000  0.000000  0.000000  0.000000  0.000000  
96  0.000000  0.000000  0.000000  0.000000  0.000000  
97  0.187778  0.000000  0.000000  0.597840  0.000000  
98  0.000000  0.000000  0.000000  0.245413  0.000000  
99  0.003283  0.000000  0.000000  1.128544  0.000000  

[100 rows x 8193 columns]

In [14]:
# 데이터셋들을 다 합친 다음엔 이런 식으로
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(4*4*512)))
...